In [1]:
# aa1 stuff
from aa1 import DataLoader
from aa1 import extract_features
from aa1 import check_output

from aa2 import Trainer
from aa2 import parallel_coordinates
from aa2.model import LSTM_fixed_len as model

In [2]:
import torch

device = torch.device('cuda:3')

In [3]:
# assigment 1 stuff
dataset = DataLoader(data_dir="../../DDICorpus/DDICorpus/", device=device)
train_y, val_y, test_y = check_output(dataset.get_y()) 
train_X, val_X, test_X = check_output(extract_features(
                                                        data=dataset.data_df,
                                                        max_sample_length=dataset.max_sample_length,
                                                        id2word=dataset.id2word,
                                                        device=device
                                                        #Add any addtional arguments here
                                                       ))

In [4]:
#set up trainer
model_dump = "/tmp/aa2_models/" #you are allowed to change the dump_folder
trainer = Trainer(dump_folder=model_dump) 

In [5]:
# define a set of hyperparamaters
# test at least 5 different sets of hyperparamaters 
set_hyperparamaters = [{"learning_rate": 0.1,
                        "hidden_size": 1000,
                        "number_layers": 3,
                        "optimizer": "adam",
                        "batch_size": 25,
                        "bidirectional": 1
                       },
                       {"learning_rate": 0.02,
                        "hidden_size": 100,
                        "number_layers": 5,
                        "optimizer": "adam",
                        "batch_size": 50,
                        "bidirectional": 1
                       }
                        ]

In [6]:
# hyperparamater tuning
# train you model with your set of hyperparamaters
for hp in set_hyperparamaters:
    trainer.train(train_X, train_y, val_X, val_y, model, hp)

Total loss in epoch 0 is 80.79725646972656.
Total loss in epoch 1 is 80.79725646972656.
Total loss in epoch 2 is 80.79725646972656.
Total loss in epoch 3 is 80.79725646972656.
Total loss in epoch 4 is 80.79725646972656.
Total loss in epoch 5 is 80.79725646972656.
Total loss in epoch 6 is 80.79725646972656.
Total loss in epoch 7 is 80.79725646972656.
Total loss in epoch 8 is 80.79725646972656.
Total loss in epoch 9 is 80.79725646972656.
Total loss in epoch 10 is 80.79725646972656.
Total loss in epoch 11 is 80.79725646972656.
Total loss in epoch 12 is 80.79725646972656.
Total loss in epoch 13 is 80.79725646972656.
Total loss in epoch 14 is 80.79725646972656.
Total loss in epoch 15 is 80.79725646972656.
Total loss in epoch 16 is 80.79725646972656.
Total loss in epoch 17 is 80.79725646972656.
Total loss in epoch 18 is 80.79725646972656.
Total loss in epoch 19 is 80.79725646972656.
Total loss in epoch 20 is 80.79725646972656.
Total loss in epoch 21 is 80.79725646972656.
Total loss in epoch 

In [7]:
# create a parallel coordination plot over hyperparamaters
# add path to models and change metric to what ever metric you have chosen to use/want to use
parallel_coordinates(model_dump, metric="loss")

models found: []


KeyError: 'loss'

In [ ]:
# Based on the Parallel Coordination plot define 3 new hyperparamaters set thay you 
# think are worth testing
set_hyperparamaters_2 = [
                        # Example:
                        # {
                        #   "learning_rate": 0.1,
                        #   "number_layers": 3,
                        #   "optimizer": "adam"
                        # }
                        ]

In [ ]:
# train models for the new hyperparamaters
for hp in set_hyperparamaters_2:
    trainer.train(train_X, train_y, val_X, val_y, YOUR_MODEL_CLASS, hp)

In [ ]:
# create a parallel coordination plot over hyperparamaters again
parallel_coordinates(model_dump, metric="loss")

In [ ]:
#choose the best model base on the last parallel coordination plot
best_model_path = "PATH TO THE BEST MODEL"
scores = trainer.test(test_X, test_y, YOUR_MODEL_CLASS, best_model_path)